In [1]:
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [0]:
from keras.applications.inception_v3 import preprocess_input

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras import backend as K

In [4]:
model = InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [5]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [0]:
K.set_learning_phase(0)

In [0]:
layer_contributions = {
    'mixed2' : 1.5,
    'mixed3' : 4.5,
    'mixed4' : 3.,
    'mixed5' : 2.5,
}

In [108]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x7f8d6b27fdd8>,
 'activation_10': <keras.layers.core.Activation at 0x7f8d64e26320>,
 'activation_11': <keras.layers.core.Activation at 0x7f8d64dd5dd8>,
 'activation_12': <keras.layers.core.Activation at 0x7f8d64e04c50>,
 'activation_13': <keras.layers.core.Activation at 0x7f8d64db9e80>,
 'activation_14': <keras.layers.core.Activation at 0x7f8d64d61e48>,
 'activation_15': <keras.layers.core.Activation at 0x7f8d64d8a2e8>,
 'activation_16': <keras.layers.core.Activation at 0x7f8d64d39da0>,
 'activation_17': <keras.layers.core.Activation at 0x7f8d64ce6c18>,
 'activation_18': <keras.layers.core.Activation at 0x7f8d64d11630>,
 'activation_19': <keras.layers.core.Activation at 0x7f8d64c55f60>,
 'activation_2': <keras.layers.core.Activation at 0x7f8d64fbaeb8>,
 'activation_20': <keras.layers.core.Activation at 0x7f8d64c85d30>,
 'activation_21': <keras.layers.core.Activation at 0x7f8d64c1ad68>,
 'activation_22': <keras.layers.core.Activation at

In [0]:
loss = K.variable(0.)

In [110]:
layer_contributions

{'mixed2': 1.5, 'mixed3': 4.5, 'mixed4': 3.0, 'mixed5': 2.5}

In [111]:
layer_dict['mixed5'].output

<tf.Tensor 'mixed5/concat:0' shape=(None, None, None, 768) dtype=float32>

In [112]:
loss = 0
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  print(coeff)
  activation = layer_dict[layer_name].output
  print(activation)
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  print(scaling)
  # print(activation[:, 2:, -2, 2:, -2, :])
  loss += coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling

1.5
Tensor("mixed2/concat:0", shape=(None, None, None, 288), dtype=float32)
Tensor("Prod_16:0", shape=(), dtype=float32)
4.5
Tensor("mixed3/concat:0", shape=(None, None, None, 768), dtype=float32)
Tensor("Prod_17:0", shape=(), dtype=float32)
3.0
Tensor("mixed4/concat:0", shape=(None, None, None, 768), dtype=float32)
Tensor("Prod_18:0", shape=(), dtype=float32)
2.5
Tensor("mixed5/concat:0", shape=(None, None, None, 768), dtype=float32)
Tensor("Prod_19:0", shape=(), dtype=float32)


In [0]:
dream = model.input

In [0]:
grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]

fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

def grad_ascent(x, iterations, step, max_loss):
  for i in range(iterations):
    loss_value, grad_value = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    print('Loss value at', i, ':', loss_value, 'Grad value', grad_value)
    x += step * grad_value
    
  return x


In [0]:
from scipy import misc

In [0]:
import scipy
from keras.preprocessing import image
from PIL import Image
import matplotlib

def resize_img(img, size):
  img = np.copy(img)
  factors = (1,
             float(size[0]) / img.shape[1],
             float(size[1]) / img.shape[2],
             1)
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  matplotlib.image.imsave(fname, deprocess_image(np.copy(img)))
  # im = Image.fromarray(img)
  # im.save("your_file.png")
  # pil_img = deprocess_image(np.copy(img))
  # misc.imsave(fname, pil_img)

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  return img
  
def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))
  else:
    x = x.reshape((x.shape[0], x.shape[1], 3))
  x /= 2.
  x += 0.5
  x *= 255.
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [117]:
original_shape

(2048, 2048)

In [121]:
import numpy as np

step = 0.1
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = 'drive/My Drive/fon.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
  shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print('Processing image shape', shape)
  img = resize_img(img, shape)
  img = grad_ascent(img,
                    iterations=iterations,
                    step=step,
                    max_loss=max_loss)
  upscaled_shrunk_original_img = resize_img(original_img, shape)

save_img(img[0], fname='final_dream.png')

Processing image shape (382, 612)
Loss value at 0 : 3.3884459 Grad value [[[[ 0.01458167 -0.00263904 -0.0125202 ]
   [ 0.0017531   0.00801696 -0.0125903 ]
   [ 0.01939705  0.03435985 -0.00100381]
   ...
   [ 0.00641392 -0.00391266  0.00153874]
   [ 0.0008672  -0.00221604 -0.00099764]
   [ 0.          0.          0.        ]]

  [[ 0.01931327  0.01256781 -0.00573307]
   [ 0.01694393  0.0398948  -0.00781967]
   [ 0.03962288  0.07913796  0.01446479]
   ...
   [ 0.01080546  0.01512447  0.00284286]
   [-0.00783618 -0.00344113 -0.00364133]
   [ 0.          0.          0.        ]]

  [[ 0.02570127  0.01990335 -0.00219095]
   [ 0.01493547  0.07856862  0.01469146]
   [ 0.02932998  0.07597385  0.02264725]
   ...
   [ 0.00379506  0.04285028  0.01649523]
   [-0.00545834  0.03299024  0.00461332]
   [ 0.          0.          0.        ]]

  ...

  [[ 0.0005816  -0.00285957 -0.00173754]
   [ 0.00143597  0.00066951 -0.0010985 ]
   [-0.00032301  0.00318204  0.00088214]
   ...
   [-0.00065426 -0.001693